In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re

# Import ADS

In [19]:
ADS_PATH = os.path.join('../resources/WECC_ADS/downloads/2032/Public Data')
ads= pd.read_csv(ADS_PATH + '/GeneratorList.csv',skiprows=2, encoding='unicode_escape')
ads = ads[ads['State'].isin(['NM', 'AZ', 'CA', 'WA', 'OR', 'ID', 'WY', 'MT', 'UT', 'SD', 'CO', 'NV', 'NE', '0', 'TX'])]
ads['Long Name'] = ads['Long Name'].astype(str)
ads['Commission Date'] = pd.to_datetime(ads['Commission Date'], format='#%Y-%m-%d#')
ads['Retirement Date'] = pd.to_datetime(ads['Retirement Date'], format='#%Y-%m-%d#')
ads['Name'] = ads['Name'].str.replace(" ", "")
ads['Name'] = ads['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x).lower())
ads['Long Name'] = ads['Long Name'].str.replace(" ", "")
ads['Long Name'] = ads['Long Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x).lower())
ads['SubType'] = ads['SubType'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x).lower())
ads.rename({'Name': 'ads_name', 'Long Name': 'ads_long_name', 'SubType': 'subtype','Commission Date':'commission_date','Retirement Date':'retirement_date','Area Name':'balancing_area'}, axis=1, inplace=True)
ads.rename(str.lower, axis='columns', inplace=True)
ads['long id'] = ads['long id'].astype(str)
ads.head(5)

generatorkey      ads_name  bus id     bus name  bus kv unit id  \
0            40  hatchsolarec   11404  HAT_DIST_PV    0.69     'PV   
1            45       copper1   11051     COPPER_G   13.80      '1   
2            54    montana4g4   11233         MPS4   13.80      '1   
3            55    montana3g3   11232         MPS3   13.80      '1   
4            56    montana2g2   11228         MPS2   13.80      '1   

   generator typeid             subtype        long id  \
0                 4     solarpvtracking     NM-57591-1   
1                 1  ctnatgasindustrial         TX-9-1   
2                 1        ctnatgasaero  TX-58562-GT-4   
3                 1        ctnatgasaero  TX-58562-GT-3   
4                 1        ctnatgasaero  TX-58562-GT-2   

                 ads_long_name  ... state county city zipcode fueltype  \
0  hatchsolarenergycenterillc1  ...    NM    NaN  NaN     NaN    Solar   
1                      copper1  ...    TX    NaN  NaN     NaN       NG   
2                   montana4g4  ...    TX    NaN  NaN     NaN       NG   
3                   montana3g3  ...    TX    NaN  NaN     NaN       NG   
4                   montana2g2  ...    TX    NaN  NaN     NaN       NG   

   technology      btm  internalid  economicpmin economicpmax  
0    Tracking  #FALSE#         NaN             0            5  
1  Industrial  #FALSE#         NaN            10           68  
2        Areo  #FALSE#         NaN            50           97  
3        Areo  #FALSE#         NaN            50           97  
4        Areo  #FALSE#         NaN            50           97  

[5 rows x 30 columns]

In [20]:
def split_long_id(row):
    if row['long id'].split('-')[0] == row['state']:
         id_list = row['long id'].split('-')[1:]
    else:
        id_list = row['long id']
    if len(id_list) > 2:
        vals=  pd.Series([id_list[0], id_list[1] + '-' + id_list[2]])
    elif len(id_list) == 2:
        vals= pd.Series([id_list[0], id_list[1]])
    elif len(id_list) == 1:
        vals= pd.Series([id_list[0], None])
    elif len(id_list) == 0:
        vals= pd.Series([None, None])
    vals.astype(str)
    return vals

# Apply the function to the 'long id' column
ads[['plant_id_ads','generator_id_ads']] = ads.apply(split_long_id, axis=1)
ads

generatorkey        ads_name  bus id     bus name  bus kv unit id  \
0               40    hatchsolarec   11404  HAT_DIST_PV    0.69     'PV   
1               45         copper1   11051     COPPER_G   13.80      '1   
2               54      montana4g4   11233         MPS4   13.80      '1   
3               55      montana3g3   11232         MPS3   13.80      '1   
4               56      montana2g2   11228         MPS2   13.80      '1   
...            ...             ...     ...          ...     ...     ...   
5030         20640    spindlehill1   70593      SPNDLE1   18.00     'G1   
5031         20641    spindlehill2   70594      SPNDLE2   18.00     'G2   
5032         20642     opalsprings   45368         OPAL   69.00      'N   
5033         20643   dciblackwater   13402     BLACKWTR  345.00     'DC   
5034         20644  clearwaterwind  623431     NEXT_COL  345.00     'W1   

      generator typeid             subtype        long id  \
0                    4     solarpvtracking     NM-57591-1   
1                    1  ctnatgasindustrial         TX-9-1   
2                    1        ctnatgasaero  TX-58562-GT-4   
3                    1        ctnatgasaero  TX-58562-GT-3   
4                    1        ctnatgasaero  TX-58562-GT-2   
...                ...                 ...            ...   
5030                 1  ctnatgasindustrial  CO-56445-GEN1   
5031                 1  ctnatgasindustrial  CO-56445-GEN2   
5032                 2               hydro     OR-54251-1   
5033                 4          dcintertie           NM--   
5034                 4           wtonshore   MT-65577-CW1   

                                      ads_long_name  ...         city zipcode  \
0                       hatchsolarenergycenterillc1  ...          NaN     NaN   
1                                           copper1  ...          NaN     NaN   
2                                        montana4g4  ...          NaN     NaN   
3                                        montana3g3  ...          NaN     NaN   
4                                        montana2g2  ...          NaN     NaN   
...                                             ...  ...          ...     ...   
5030                    spindlehillenergycentergen1  ...  Fort Lupton     NaN   
5031                    spindlehillenergycentergen2  ...  Fort Lupton     NaN   
5032  deschutesvalleywaterdistrictopalsprings1hy593  ...          NaN     NaN   
5033          blackwaterdcintertiefictionalresource  ...          NaN     NaN   
5034                                 clearwaterwind  ...          NaN     NaN   

     fueltype  technology      btm internalid  economicpmin  economicpmax  \
0       Solar    Tracking  #FALSE#        NaN             0             5   
1          NG  Industrial  #FALSE#        NaN            10            68   
2          NG        Areo  #FALSE#        NaN            50            97   
3          NG        Areo  #FALSE#        NaN            50            97   
4          NG        Areo  #FALSE#        NaN            50            97   
...       ...         ...      ...        ...           ...           ...   
5030       NG  Industrial  #FALSE#        NaN            81           160   
5031       NG  Industrial  #FALSE#        NaN            80           160   
5032    Water         NaN  #FALSE#        NaN             0             5   
5033      NaN         NaN  #FALSE#        NaN             0           200   
5034     Wind     Onshore  #FALSE#        NaN             0           750   

      plant_id_ads generator_id_ads  
0            57591                1  
1                9                1  
2            58562             GT-4  
3            58562             GT-3  
4            58562             GT-2  
...            ...              ...  
5030         56445             GEN1  
5031         56445             GEN2  
5032         54251                1  
5033                                 
5034         65577              CW1  

[4254 rows x 32 c

In [21]:
# ads['eia_id_ads'] = ads['long id'].str.split('-').str[1].fillna(99999)
# ads['generator_id_ads'] = ads['long id'].str.split('-').str[2].fillna(99999).astype(str)
# ads['eia_id_ads'] = ads['eia_id_ads'].replace('N/A', '9999').replace('??', '9999').replace(['',], '9999')
# ads.loc[~ads['eia_id_ads'].astype(str).str.isdigit(),['eia_id_ads']] = 99999  #replace all non integer values with 9999
# #TODO: THERE IS a wind plant (id: 57621) with three eia_ids that need to be split up since they exist.
# # ads.iloc[4864] #issue with this row
# # ads[~ads['eia_id_ads'].astype(str).str.isdigit()][['servicestatus', 'commission_date', 'retirement_date', 'devstatus','maxcap(mw)','eia_id_ads']].sort_values(by='maxcap(mw)', ascending=False).head(30)
# # ads.loc[ads[~ads['eia_id_ads'].astype(str).str.isdigit()]['maxcap(mw)'].sort_values(ascending=False).head(20).index]
# ads['eia_id_ads'] = ads['eia_id_ads'].astype(int).astype(str)
# # ads['gen_id_ads'] = ads['long id'].str.split('-').str[2].astype(str)
# ads.head(5)


## 2032 Network Preparation
### Populate ADS data with lat lon

In [114]:
eia_ads_mapping = pd.read_csv('../repo_data/eia_mappings/eia_ads_generator_mapping.csv')
lat_dict = dict(zip(eia_ads_mapping['GeneratorKey'], eia_ads_mapping['latitude']))
lon_dict = dict(zip(eia_ads_mapping['GeneratorKey'], eia_ads_mapping['longitude']))
eia_id_dict = dict(zip(eia_ads_mapping['GeneratorKey'], eia_ads_mapping['plant_id_eia']))
# combine lat_dict and lon_dict into one dictionary
lat_lon_dict = {k: (lat_dict[k], lon_dict[k]) for k in lat_dict}
ads['lat'] = ads['generatorkey'].map(lat_lon_dict).str[0]
ads['lon'] = ads['generatorkey'].map(lat_lon_dict).str[1]
ads['plant_id_eia'] = ads['generatorkey'].map(eia_id_dict)
print('Total Num ADS Gens: ', ads.shape[0])
print('Number of ads with missing lat long: ', ads['lat'].isna().sum())
ads_matched = ads[~(ads['lat'].isnull() | ads['lon'].isnull())]
ads_matched
# bus_locs = ads.groupby('bus id').agg({'lat': 'mean', 'lon': 'mean'}).reset_index().dropna()
# ads[['lat', 'lon']] = ads[['lat', 'lon']].fillna(ads['bus id'].map(bus_locs.set_index('bus id')['lat']))
# ads[['lat', 'lon']] = ads[['lat', 'lon']].fillna(ads['bus id'].map(bus_locs.set_index('bus id')['lon']))
# print('Number of ads with missing lat long: ', ads['lat'].isna().sum())
# #didnt do anything

Total Num ADS Gens:  4254
Number of ads with missing lat long:  1567


generatorkey         ads_name  bus id     bus name  bus kv unit id  \
0               40     hatchsolarec   11404  HAT_DIST_PV    0.69     'PV   
6              175        roosevelt   15951     RSVLTGEN   13.80      '1   
25             518       chulavista   22617       OY GEN   13.80      '1   
26             530     borderpeaker   22149     CALPK_BD   13.80      '1   
27             531  enterprisepeakr   22153     CALPK_ES   13.80      '1   
...            ...              ...     ...          ...     ...     ...   
5026         20636        niyolwind   72739    NIYOL_GEN    0.63      'W   
5030         20640     spindlehill1   70593      SPNDLE1   18.00     'G1   
5031         20641     spindlehill2   70594      SPNDLE2   18.00     'G2   
5032         20642      opalsprings   45368         OPAL   69.00      'N   
5034         20644   clearwaterwind  623431     NEXT_COL  345.00     'W1   

      generator typeid             subtype         long id  \
0                    4     solarpvtracking      NM-57591-1   
6                    2               hydro     AZ-149-ROOS   
25                   1        ctnatgasaero   CA-55540-GEN1   
26                   1  ctnatgasindustrial   CA-55510-CPP4   
27                   1  ctnatgasindustrial   CA-55513-CPP7   
...                ...                 ...             ...   
5026                 4           wtonshore  CO-65059-WNIYO   
5030                 1  ctnatgasindustrial   CO-56445-GEN1   
5031                 1  ctnatgasindustrial   CO-56445-GEN2   
5032                 2               hydro      OR-54251-1   
5034                 4           wtonshore    MT-65577-CW1   

                                      ads_long_name  ...  technology      btm  \
0                       hatchsolarenergycenterillc1  ...    Tracking  #FALSE#   
6                                     rooseveltroos  ...         NaN  #FALSE#   
25                                  chulavistaigen1  ...        Areo  #FALSE#   
26                calpeakpowerborderpeakerplantcpp4  ...  Industrial  #FALSE#   
27            calpeakpowerenterprisepeakerplantcpp7  ...  Industrial  #FALSE#   
...                                             ...  ...         ...      ...   
5026                                   niyolwindllc  ...     Onshore  #FALSE#   
5030                    spindlehillenergycentergen1  ...  Industrial  #FALSE#   
5031                    spindlehillenergycentergen2  ...  Industrial  #FALSE#   
5032  deschutesvalleywaterdistrictopalsprings1hy593  ...         NaN  #FALSE#   
5034                                 clearwaterwind  ...     Onshore  #FALSE#   

     internalid economicpmin economicpmax plant_id_ads  generator_id_ads  \
0           NaN            0            5        57591                 1   
6           NaN            0           36          149              ROOS   
25          NaN        4.375           35        55540              GEN1   
26          NaN           44           48        55510              CPP4   
27          NaN            6           48        55513              CPP7   
...         ...          ...          ...          ...               ...   
5026        NaN            0          200        65059             WNIYO   
5030        NaN           81          160        56445              GEN1   
5031        NaN           80          160        56445              GEN2   
5032        NaN            0            5        54251                 1   
5034        NaN            0          750        65577               CW1   

            lat       lon plant_id_eia  
0     32.630000 -107.2597      57591.0  
6     33.671104 -111.1618        149.0  
25    32.591667 -117.0586      55540.0  
26    32.562448 -116.9439      55510.0  
27    33.121943 -117.1174      55513.0  
...         ...       ...          ...  
5026  40.613364 -102.9526      65059.0  
5030  40.092192 -104.8878      56445.0  
5031  40.092192 -104.8878      56445.0  
5032  44.490040 -121.2988      54251.0  
5034  46.81

In [126]:
#Trying other methods to match non-direct matches
ads_unmatched = ads[(ads['lat'].isnull() | ads['lon'].isnull())]
print(ads_unmatched.columns)
ads_unmatched = ads_unmatched[ads_unmatched.btm == '#FALSE#'] #TODO: filtering out BTM resources for now
eia_loc.plant_id_eia = eia_loc.plant_id_eia.astype(str)
matched_2 = pd.merge(ads_unmatched,eia_loc,left_on= 'plant_id_ads', right_on='plant_id_eia', how = 'inner')
matched_2 = matched_2.drop(columns=['lat','lon']).rename(columns={'Latitude':'lat','Longitude':'lon'})
ads_matched_2 = pd.concat([ads_matched, matched_2])
ads_unmatched = ads_unmatched[~ads_unmatched.generatorkey.isin(matched_2.generatorkey)]
# ads_unmatched.to_csv('ads_unmatched.csv')
ads_eia_mapping = ads_matched_2[['generatorkey','ads_name','long id', 'bus id', 'bus name', 'devstatus', 'balancing_area', 'region name', 'plant_id_ads','plant_id_eia', 'generator_id_ads', 'lat', 'lon']]
ads_eia_mapping.to_csv('/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/repo_data/eia_mappings/eia_ads_generator_mapping_updated.csv',index=True)
ads_eia_mapping

Index(['generatorkey', 'ads_name', 'bus id', 'bus name', 'bus kv', 'unit id',
       'generator typeid', 'subtype', 'long id', 'ads_long_name',
       'servicestatus', 'commission_date', 'retirement_date', 'devstatus',
       'balancing_area', 'region name', 'mincap(mw)', 'maxcap(mw)',
       'initialdispatch(mw)', 'save to binary', 'state', 'county', 'city',
       'zipcode', 'fueltype', 'technology', 'btm', 'internalid',
       'economicpmin', 'economicpmax', 'plant_id_ads', 'generator_id_ads',
       'lat', 'lon', 'plant_id_eia'],
      dtype='object')


generatorkey         ads_name        long id  bus id      bus name  \
0              40     hatchsolarec     NM-57591-1   11404   HAT_DIST_PV   
6             175        roosevelt    AZ-149-ROOS   15951      RSVLTGEN   
25            518       chulavista  CA-55540-GEN1   22617        OY GEN   
26            530     borderpeaker  CA-55510-CPP4   22149      CALPK_BD   
27            531  enterprisepeakr  CA-55513-CPP7   22153      CALPK_ES   
..            ...              ...            ...     ...           ...   
131         17615    sunshinehydro         4514_1   70261       LEGGETT   
132         17607            paris          830_1   66151          OVID   
133         20611  orchardwindfarm  OR-62935-OCHW   44586  ORCHARD WNDL   
134         20631      eonvalencia     AZ-59220-1  160635    IRVINGTON1   
135         20632   ironhorsesolar      AZ-60633-  160698         VAIL2   

             devstatus balancing_area region name plant_id_ads  plant_id_eia  \
0             Existing            EPE      SW_EPE        57591       57591.0   
6             Existing            SRP      SW_SRP          149         149.0   
25            Existing           CISD     CA_CISO        55540       55540.0   
26            Existing           CISD     CA_CISO        55510       55510.0   
27            Existing           CISD     CA_CISO        55513       55513.0   
..                 ...            ...         ...          ...           ...   
131           Existing           PSCO     RM_PSCO            4           NaN   
132           Existing           PAID     BS_PACE            8           NaN   
133  Future-Conceptual           PACW     NW_PACW        62935           NaN   
134           Existing           TEPC     SW_TEPC        59220           NaN   
135           Existing           TEPC     SW_TEPC        60633           NaN   

    generator_id_ads        lat        lon  
0                  1      32.63  -107.2597  
6               ROOS  33.671104  -111.1618  
25              GEN1  32.591667  -117.0586  
26              CPP4  32.562448  -116.9439  
27              CPP7  33.121943  -117.1174  
..               ...        ...        ...  
131              5-1  32.583889 -86.283056  
132              3-0  33.644344 -87.196486  
133             OCHW      45.38  -119.3332  
134                1  39.719364 -76.161625  
135                      34.499    -78.653  

[2823 rows x 13 columns]

## Historical Network Preparation
### Populate EIA data with thermal ramp rates

In [172]:
#return datatype of each column in a eia_plant_locs
print(eia_plants_locs.dtypes)
eia_plants_locs.


Plant Code            float64
plant_name_eia         object
generator_id           object
Technology             object
Prime Mover            object
capacity_mw            object
summer_capacity_mw     object
winter_capacity_mw     object
p_nom_min              object
Status                 object
Operating Year         object
Energy Source 1        object
tech_type              object
fuel_type              object
prime_mover            object
plant_id_eia            int64
State                  object
Latitude               object
Longitude              object
dtype: object


In [173]:
# eia_plants_locs.groupby('plant_id_eia').agg({'capacity_mw': 'mean', 'summer_capacity_mw': 'mean'})

In [156]:
eia_plants_locs_thermal = eia_plants_locs[~eia_plants_locs.tech_type.isin(['Solar','Hydro','Wind','Storage',])]
eia_plants_locs_thermal

Plant Code         plant_name_eia generator_id  \
22            9.0                 Copper            1   
125          59.0                 Platte            1   
126          60.0   Whelan Energy Center            1   
127          60.0   Whelan Energy Center            2   
226          99.0           Frederickson            1   
...           ...                    ...          ...   
25327     66614.0  OTHERCitizensHospital        CTZH3   
25328     66614.0  OTHERCitizensHospital        CTZH4   
25329     66614.0  OTHERCitizensHospital        CTZH5   
25330     66614.0  OTHERCitizensHospital        CTZH6   
25331     66614.0  OTHERCitizensHospital        CTZH7   

                                   Technology Prime Mover capacity_mw  \
22       Natural Gas Fired Combustion Turbine          GT        86.9   
125                   Conventional Steam Coal          ST       109.8   
126                   Conventional Steam Coal          ST        76.3   
127                   Conventional Steam Coal          ST         248   
226      Natural Gas Fired Combustion Turbine          GT        88.9   
...                                       ...         ...         ...   
25327  Natural Gas Internal Combustion Engine          IC         0.4   
25328  Natural Gas Internal Combustion Engine          IC         0.4   
25329  Natural Gas Internal Combustion Engine          IC         0.4   
25330  Natural Gas Internal Combustion Engine          IC         0.4   
25331  Natural Gas Internal Combustion Engine          IC         0.4   

      summer_capacity_mw winter_capacity_mw p_nom_min Status Operating Year  \
22                    63                 65         3     OP           1980   
125                  100                100        40     OP           1982   
126                   76                 76        38     OP           1981   
127                  220                220       125     OP           2011   
226                   67               81.1         5     OP           1981   
...                  ...                ...       ...    ...            ...   
25327                0.4                0.4         0     OP           2020   
25328                0.4                0.4         0     OP           2020   
25329                0.4                0.4         0     OP           2020   
25330                0.4                0.4         0     OP           2020   
25331                0.4                0.4         0     OP           2020   

      Energy Source 1 tech_type fuel_type prime_mover plant_id_eia State  \
22                 NG    Gas_SC       Gas          CT            9    TX   
125               SUB      Coal      Coal          CT           59    NE   
126               SUB      Coal      Coal          CT           60    NE   
127               SUB      Coal      Coal          CT           60    NE   
226                NG    Gas_SC       Gas          CT           99    WA   
...               ...       ...       ...         ...          ...   ...   
25327              NG    Gas_SC       Gas          CT        66614    TX   
25328              NG    Gas_SC       Gas          CT        66614    TX   
25329              NG    Gas_SC       Gas          CT        66614    TX   
25330              NG    Gas_SC       Gas          CT        66614    TX   
25331              NG    Gas_SC       Gas          CT        66614    TX   

        Latitude  Longitude  
22       31.7569   -106.375  
125    40.854765 -98.348222  
126    40.580872 -98.312437  
127    40.580872 -98.312437  
226    47.079722   -122.365  
...          ...        ...  
25327  28.812804 -96.977765  
25328  28.812804 -96.977765  
25329  28.812804 -96.977765  
25330  28.812804 -96.977765  
25331  28.812804 -96.977765  

[4262 rows x 19 columns]

In [159]:
pd.merge(eia_plants_locs_thermal, ads_eia_mapping, how='inner', left_on=['Plant Code','generator_id'], right_on=['plant_id_eia','generator_id_ads'])

Plant Code                            plant_name_eia generator_id  \
0          99.0                              Frederickson            1   
1          99.0                              Frederickson            2   
2         114.0                                   Douglas            1   
3         116.0                                  Ocotillo          GT1   
4         116.0                                  Ocotillo          GT2   
..          ...                                       ...          ...   
723     60698.0  Stanton Energy Reliability Center Hybrid          GT2   
724     61204.0                      AES ES ALAMITOS, LLC        ALMTS   
725     61366.0                    Top Gun Energy Storage         TGES   
726     63834.0             Gateway Energy Storage System        GTWAY   
727     64103.0                     Desert Harvest II LLC        DH003   

                               Technology Prime Mover capacity_mw  \
0    Natural Gas Fired Combustion Turbine          GT        88.9   
1    Natural Gas Fired Combustion Turbine          GT        88.9   
2                       Petroleum Liquids          GT        26.1   
3    Natural Gas Fired Combustion Turbine          GT        53.1   
4    Natural Gas Fired Combustion Turbine          GT        53.1   
..                                    ...         ...         ...   
723  Natural Gas Fired Combustion Turbine          GT        70.5   
724                             Batteries          BA         100   
725                             Batteries          BA          30   
726                             Batteries          BA         250   
727                             Batteries          BA          35   

    summer_capacity_mw winter_capacity_mw p_nom_min Status  ... bus id  \
0                   67               81.1         5     OP  ...  42721   
1                   67               81.1         5     OP  ...  42722   
2                   15                 19         2     OP  ...  85831   
3                   50                 62         4     OP  ...  14921   
4                   50                 62         4     OP  ...  14922   
..                 ...                ...       ...    ...  ...    ...   
723               70.5               70.5         0     OP  ...  24016   
724                100                100         0     OP  ...  86033   
725                 30                 30        30     OP  ...  23412   
726                250                250         0     OP  ...  23710   
727                 35                 35         1     OP  ...  25036   

       bus name           devstatus balancing_area region name plant_id_ads  \
0      FREDERC1            Existing           PSEI     NW_PSEI           99   
1      FREDERC2            Existing           PSEI     NW_PSEI           99   
2      FAIRVWGN            Existing           AZPS     SW_AZPS          114   
3       OCOTGT1            Existing           AZPS     SW_AZPS          116   
4       OCOTGT2            Existing           AZPS     SW_AZPS          116   
..          ...                 ...            ...         ...          ...   
723       BARRE  Under Construction           CISC     CA_CISO        60698   
724   APFWG Bus            Existing           CISC     CA_CISO        61204   
725     Q1434_G            Existing           CISD     CA_CISO        61366   
726  Q1170_BESS            Existing           CISD     CA_CISO        63834   
727  DSTHVST_G2            Existing           CISC     CA_CISO        64103   

    plant_id_eia_y generator_id_ads        lat       lon  
0             99.0                1  47.079722  -122.435  
1             99.0                2  47.079722  -122.435  
2            114.0                1  31.364122 -109.5538  
3            116.0              GT1    33.4225 -111.9122  
4            116.0              GT2    33.4225 -111.9122  
..             ...              ...        ...       ...  
723        60698.0              GT2 

In [143]:
ads_eia_mapping.head(5)

generatorkey         ads_name        long id  bus id     bus name  \
0             40     hatchsolarec     NM-57591-1   11404  HAT_DIST_PV   
6            175        roosevelt    AZ-149-ROOS   15951     RSVLTGEN   
25           518       chulavista  CA-55540-GEN1   22617       OY GEN   
26           530     borderpeaker  CA-55510-CPP4   22149     CALPK_BD   
27           531  enterprisepeakr  CA-55513-CPP7   22153     CALPK_ES   

   devstatus balancing_area region name plant_id_ads  plant_id_eia  \
0   Existing            EPE      SW_EPE        57591       57591.0   
6   Existing            SRP      SW_SRP          149         149.0   
25  Existing           CISD     CA_CISO        55540       55540.0   
26  Existing           CISD     CA_CISO        55510       55510.0   
27  Existing           CISD     CA_CISO        55513       55513.0   

   generator_id_ads        lat       lon  
0                 1      32.63 -107.2597  
6              ROOS  33.671104 -111.1618  
25             GEN1  32.591667 -117.0586  
26             CPP4  32.562448 -116.9439  
27             CPP7  33.121943 -117.1174

In [145]:
ads_thermal= pd.read_csv(ADS_PATH + '/Thermal_General_Info.csv',skiprows=1, )#encoding='unicode_escape')
ads_thermal.head(5)

GeneratorName   Turbine Type Model Type  Co-gen  Installed Capacity(MW)  \
0      Copper_1             GT        NaN   False                    68.0   
1  Montana_4_G4  GT LMS_LMS100        NaN   False                    97.0   
2  Montana_3_G3  GT LMS_LMS100        NaN   False                    97.0   
3  Montana_2_G2  GT LMS_LMS100        NaN   False                    97.0   
4  Montana_1_G1  GT LMS_LMS100        NaN   False                    93.2   

   MustRun  QuickStart           Fuel Name Secondary Fuel Name  IfUseBid  ...  \
0    False       False  NG_West Texas_Waha         DefaultFuel     False  ...   
1    False       False  NG_West Texas_Waha         DefaultFuel     False  ...   
2    False       False  NG_West Texas_Waha         DefaultFuel     False  ...   
3    False       False  NG_West Texas_Waha         DefaultFuel     False  ...   
4    False       False  NG_West Texas_Waha         DefaultFuel     False  ...   

  Generic Startup Name  Startup Cost Fixed($)   Startup Fuel Name  \
0                  NaN               3777.897  NG_West Texas_Waha   
1                  NaN               2672.257  NG_West Texas_Waha   
2                  NaN               2672.257  NG_West Texas_Waha   
3                  NaN               2642.299  NG_West Texas_Waha   
4                  NaN               2642.299  NG_West Texas_Waha   

   StartFuel(MMBTu) Alpha  Startup Time  Generic VOM Name VOM Cost  STCAP  \
0            17.721     1             1               NaN     0.56    168   
1           453.000     1             1               NaN     0.45    168   
2           453.000     1             1               NaN     0.45    168   
3           453.000     1             1               NaN     0.45    168   
4           453.000     1             1               NaN     0.45    168   

   Estimated CF  
0           0.6  
1           0.6  
2           0.6  
3           0.6  
4           0.6  

[5 rows x 31 columns]

In [141]:
merged_thermal = pd.merge(eia_plants_locs, ads_eia_mapping[['plant_id_ads','generator_id_ads']], left_on=['plant_id_eia','generator_id'], right_on=['plant_id_ads','generator_id_ads'], how='left')
print(merged_thermal.columns)
merged_thermal

Index(['Plant Code', 'plant_name_eia', 'generator_id', 'Technology',
       'Prime Mover', 'capacity_mw', 'summer_capacity_mw',
       'winter_capacity_mw', 'p_nom_min', 'Status', 'Operating Year',
       'Energy Source 1', 'tech_type', 'fuel_type', 'prime_mover',
       'plant_id_eia', 'State', 'Latitude', 'Longitude', 'plant_id_ads',
       'generator_id_ads'],
      dtype='object')


Plant Code         plant_name_eia generator_id  \
0            9.0                 Copper            1   
1           34.0                Rollins           1P   
2           59.0                 Platte            1   
3           60.0   Whelan Energy Center            1   
4           60.0   Whelan Energy Center            2   
...          ...                    ...          ...   
8254     66614.0  OTHERCitizensHospital        CTZH3   
8255     66614.0  OTHERCitizensHospital        CTZH4   
8256     66614.0  OTHERCitizensHospital        CTZH5   
8257     66614.0  OTHERCitizensHospital        CTZH6   
8258     66614.0  OTHERCitizensHospital        CTZH7   

                                  Technology Prime Mover capacity_mw  \
0       Natural Gas Fired Combustion Turbine          GT        86.9   
1                 Conventional Hydroelectric          HY        12.1   
2                    Conventional Steam Coal          ST       109.8   
3                    Conventional Steam Coal          ST        76.3   
4                    Conventional Steam Coal          ST         248   
...                                      ...         ...         ...   
8254  Natural Gas Internal Combustion Engine          IC         0.4   
8255  Natural Gas Internal Combustion Engine          IC         0.4   
8256  Natural Gas Internal Combustion Engine          IC         0.4   
8257  Natural Gas Internal Combustion Engine          IC         0.4   
8258  Natural Gas Internal Combustion Engine          IC         0.4   

     summer_capacity_mw winter_capacity_mw p_nom_min Status  ...  \
0                    63                 65         3     OP  ...   
1                  12.1               12.1       0.5     OP  ...   
2                   100                100        40     OP  ...   
3                    76                 76        38     OP  ...   
4                   220                220       125     OP  ...   
...                 ...                ...       ...    ...  ...   
8254                0.4                0.4         0     OP  ...   
8255                0.4                0.4         0     OP  ...   
8256                0.4                0.4         0     OP  ...   
8257                0.4                0.4         0     OP  ...   
8258                0.4                0.4         0     OP  ...   

     Energy Source 1 tech_type fuel_type prime_mover plant_id_eia State  \
0                 NG    Gas_SC       Gas          CT            9    TX   
1                WAT     Hydro       NaN          HY           34    CA   
2                SUB      Coal      Coal          CT           59    NE   
3                SUB      Coal      Coal          CT           60    NE   
4                SUB      Coal      Coal          CT           60    NE   
...              ...       ...       ...         ...          ...   ...   
8254              NG    Gas_SC       Gas          CT        66614    TX   
8255              NG    Gas_SC       Gas          CT        66614    TX   
8256              NG    Gas_SC       Gas          CT        66614    TX   
8257              NG    Gas_SC       Gas          CT        66614    TX   
8258              NG    Gas_SC       Gas          CT        66614    TX   

       Latitude   Longitude plant_id_ads generator_id_ads  
0       31.7569    -106.375            9                1  
1     39.134259 -120.953341           34               1P  
2     40.854765  -98.348222          NaN              NaN  
3     40.580872  -98.312437          NaN              NaN  
4     40.580872  -98.312437          NaN              NaN  
...         ...         ...          ...              ...  
8254  28.812804  -96.977765          NaN              NaN  
8255  28.812804  -96.977765          NaN              NaN  
8256  28.812804  -96.977765          NaN              NaN  
8257  28.812804  -96.977765          NaN              NaN  
8258  28.812804  -96.977765          NaN              NaN  

[8259 rows x 21 columns]

In [ ]:
pd.merge(merged_thermal)

# Loading PuDl Data

url_gens_860 = 'https://data.catalyst.coop/pudl/generators_eia860.csv?_stream=on&report_date__gte=2022-01-01&_sort_desc=report_date&_labels=on&_size=max'
url_gen_entity_860 = 'https://data.catalyst.coop/pudl/generators_entity_eia.csv?_stream=on&_labels=on&_size=max'
url_plants_860 = 'https://data.catalyst.coop/pudl/plants_eia860.csv?_stream=on&report_date__gte=2022-01-01&_labels=on&_size=max'
url_plants_entitiy ='https://data.catalyst.coop/pudl/plants_entity_eia.csv?_stream=on&_size=max'
# os.mkdir('/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/data/pudl')
PATH = '/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/'
gens = pd.read_csv(url_gens_860)
gens.report_date = pd.to_datetime(gens.report_date)